# Constrained Fine Structure Fitting on TbScO3
This notebook shows how to perform the constrained fine structure fit on EELS datasets. The example shows an atomic scale map on TbScO3 where three different methods are applied to the data.
1. Unconstrained fine structure: No constrains are applied to the fine structure
2. Constrained fine structure: Constrains derived from the Bethe sum rule are applied to the three core-loss edges
3. Power-law subtraction which is the conventional EELS processing method. 

For the model based methods, the constrained background model is also used. For more information on the methodology see the work of [Jannis et al](https://arxiv.org/abs/2408.11870).


In [1]:
%matplotlib qt

In [2]:
import pyEELSMODEL.api as em
import matplotlib.pyplot as plt
import os
import numpy as np

from pyEELSMODEL.components.linear_background import LinearBG
from pyEELSMODEL.components.gdoslin import GDOSLin
from pyEELSMODEL.components.constrained_gdoslin import ConstrainedGDOSLin
from pyEELSMODEL.components.MScatter.mscatterfft import MscatterFFT

In [4]:
def do_fine_structure_fit(s, ll, elements, edges, onsets, ewidths, ns, settings, method):
    """
    Small function which wraps the fitting of the multispectrum or a single spectrum. 

    Args:
        s (MultiSpectrum): The multispectrum which needs to be fitted. 
        ll (MultiSpectrum): The low-loss multispectrum
        elements (list): List of elements which are present in the spectrum
        edges (list): List of edges which are present in the spectrum,
            should match with elements
        onsets (list): Onset energies of the atomic cross sections
        ewidths (list): The extend of the fine structures
        ns (list): The number of parameters for each fine structure
        settings (list): E0 (V), alpha (radians), beta (radians)
        method (str): Indicate if fine structure should be constrained or not
    """
    
    specshape=s.get_spectrumshape()

    E0 = settings[0]
    alpha = settings[1]
    beta = settings[2]

    #the background is modelled with a power op 3 for convenience
    BG = LinearBG(specshape, rlist=[1,2,3,4,5])
    BG.use_approx = "sufficient"

    comp_elements = []
    comp_fine = []
    for edge, element, onset, interval, n in zip(edges, elements, onsets, ewidths, ns):
        # comp = HydrogenicCoreLossEdge(specshape, 1, E0, alpha, beta, element, edge, eshift=onset)

        comp = em.ZezhongCoreLossEdgeCombined(specshape, 1, E0, alpha, beta, 
                                              element, edge, eshift=onset, q_steps=20)

        comp_elements.append(comp)
        if method == 'constrained':
            fine = ConstrainedGDOSLin.gdoslin_from_edge(specshape, comp, ewidth=interval,degree=n,
                                interpolationtype='nearest', pre_e=0)
        else:
            fine = GDOSLin.gdoslin_from_edge(specshape, comp, ewidth=interval,degree=n,
                                            interpolationtype='nearest', pre_e=0)

        comp_fine.append(fine)
        
    llcomp = MscatterFFT(specshape, ll)
    components = [BG] + comp_elements + comp_fine + [llcomp]
    Omod = em.Model(specshape, components=components)
    fit = em.QuadraticFitter(s, Omod)  # The fitter object
    
    if type(s) is em.MultiSpectrum:
        fit.multi_fit()
        fitsig = fit.model_to_multispectrum()
        fig, maps, name = fit.show_map_result(comp_elements)

        return fit, fitsig, maps

    else:
        fit.perform_fit()
        fit.set_fit_values()
        return fit


### Dataset
The dataset can be retrieved from 

In [5]:

dir = r''

In [6]:
hl = em.MultiSpectrum.load(os.path.join(dir, 'tbsco3_coreloss.hdf5'))
ll = em.MultiSpectrum.load(os.path.join(dir, 'tbsco3_lowloss.hdf5'))

In [7]:
em.MultiSpectrumVisualizer([hl])

c:\users\daen.jannis1\documents\pyeelsmodel\pyEELSMODEL\operators\multispectrumvisualizer.py:119: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax[1].legend()


In [11]:
alpha = 28e-3
E0 = 300e3
beta= 32e-3
settings = [E0, alpha, beta]

elements = ['Sc', 'O', 'Tb']
edges = ['L', 'K', 'M']
onsets = [-2, 0, 0]

intervals = [100, 100, 200] 
ns = [30,30,50]

In [8]:
single_fitn = do_fine_structure_fit(hl.mean(), ll.mean(), elements, edges, onsets, intervals, ns, settings, None)
single_fitc = do_fine_structure_fit(hl.mean(), ll.mean(), elements, edges, onsets, intervals, ns, settings, "constrained")


cannot use analytical gradients since a convolutor is inside model


Convergent incoming beam is used
convergence angle is: 0.028
collection angle is: 0.032
Convergent incoming beam is used
convergence angle is: 0.028
collection angle is: 0.032


cannot use analytical gradients since a convolutor is inside model


Convergent incoming beam is used
convergence angle is: 0.028
collection angle is: 0.032


In [9]:
fig = single_fitn.plot()
fig = single_fitc.plot()

In [12]:
n = 256

In [13]:
fitc, fitsigc, mapsc = do_fine_structure_fit(hl[:n,:n,:], ll[:n,:n,:], elements, edges, onsets,
                                             intervals, ns, settings, "constrained")

Convergent incoming beam is used
convergence angle is: 0.028
collection angle is: 0.032
Convergent incoming beam is used
convergence angle is: 0.028
collection angle is: 0.032
Convergent incoming beam is used
convergence angle is: 0.028
collection angle is: 0.032


cannot use analytical gradients since a convolutor is inside model
65536it [17:57, 60.85it/s]
65536it [00:54, 1197.96it/s]


In [14]:
fitn, fitsign, mapsn = do_fine_structure_fit(hl[:n,:n,:], ll[:n,:n,:], elements, edges, onsets,
                                             intervals, ns, settings, None)


cannot use analytical gradients since a convolutor is inside model
65536it [18:34, 58.81it/s]
65536it [01:04, 1018.97it/s]


### 3. Power-law subtraction
The next step is to perform the background subtraction, the background windows and integration windows are defined by the user itself.

In [15]:
bg_win = [[310, 390],[480, 520],[1000,1220]] #are defined by user
int_win = [[395,500], [530,650],[1230,2000]] #are defined by user
int_maps = np.zeros((3, n, n))
bgs = []
for ii, win in enumerate(bg_win):
    rem = em.BackgroundRemoval(hl[:n,:n,:], win)
    bg = rem.calculate_multi()    
    int_maps[ii] = rem.quantify_from_edge(int_win[ii], elements[ii], edges[ii], E0, alpha, beta)
    
    bgs.append(bg)

True


0it [00:00, ?it/s]

65536it [00:09, 7117.19it/s]
65536it [02:05, 523.27it/s]
65536it [00:03, 21041.51it/s]


True


65536it [00:06, 9556.31it/s] 
65536it [05:33, 196.45it/s]
65536it [00:02, 22546.51it/s]


True


65536it [00:07, 8585.07it/s]
65536it [24:55, 43.82it/s]
65536it [00:02, 22096.57it/s]


In [16]:
mastermap = [mapsn, mapsc, int_maps]
fig, ax = plt.subplots(3, len(mastermap))

for jj, mp in enumerate(mastermap):
    for ii in range(mp.shape[0]):
        ax[ii, jj].imshow(mp[ii], cmap='viridis')
        if jj == 0:
            ax[ii,jj].set_title(f'{elements[ii]} {edges[ii]}')
        ax[ii, jj].axis('off')

In [17]:
fig, ax= plt.subplots(1, 3, figsize=(14,4))
bins_min = [.01,0.01,0.01]
bins_max = [2,8, 2.]
fs=14
mth = ['{c}', '{1}', '{2}', '{3}', '{in}']
colors = ['red', 'blue', 'green', 'orange', 'purple']

for jj, mp in enumerate(mastermap):
    for ii in range(mp.shape[0]):
        bins=np.linspace(bins_min[ii], bins_max[ii], 300)
        xx = (bins[:-1] + bins[1:])/2
        # hsq = np.histogram(mapssq[ii].flatten(), bins=bins)
        sig = mp[ii].flatten()
        sig[sig<0] = 0
        boolean = sig == 0
        sigg = sig[np.invert(boolean)]
        hl = np.histogram(sigg, bins=bins)
        avg = np.nanmean(sigg)
        std = np.nanstd(sigg)    
    
    
        print((sig==0).sum()/sig.size)
    
        label0 = r'$\langle {0} \rangle_{1}$ = {2}  $\pm$ {3}'.format(elements[ii], mth[jj], np.round(avg,2), np.round(std,2))
    
        ax[ii].fill_between(xx, hl[0],  color=colors[jj], alpha=0.5)

        ax[ii].plot(xx, hl[0], color=colors[jj],label=label0)
        

        # ax[ii].set_title(elements[ii], fontsize=fs)
        
        
lbs=[r'\textbf{a)}', r'\textbf{b)}', r'\textbf{c)}']

for ii, axe in enumerate(ax):
    axe.legend(fontsize=12)
    axe.set_xlabel(elements[ii] + r' counts [a.u.]', fontsize=fs)
    axe.set_xlim([bins_min[ii], bins_max[ii]])
    axe.set_ylim([0, None])


0.163787841796875
0.096649169921875
0.108062744140625
0.008331298828125
0.0162200927734375
0.0047454833984375
0.012054443359375
0.361358642578125
0.0456695556640625
